In [ ]:
import json
from collections import defaultdict
from IPython.display import display_png, Image

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from random import Random

In [ ]:
# Suppress warnings from Plotly.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load and filter GeoJSON file.
with open('../data/geojson-counties-fips.json') as f:
    geo = json.load(f)

STATE_UTAH = 49
geo['features'] = [feature for feature in geo['features'] if feature['properties']['STATE'] == str(STATE_UTAH)]
counties = [feature['id'] for feature in geo['features']]
county_names = {feature['id']: feature['properties']['NAME'] for feature in geo['features']}

In [ ]:
# Load adjacency data.
nbrs = defaultdict(list)

for index, row in pd.read_csv('../data/county_adjacency2024.txt', delimiter='|').iterrows():
    u = row['County GEOID']
    v = row['Neighbor GEOID']

    if u // 1000 == STATE_UTAH and v // 1000 == STATE_UTAH and u != v:
        nbrs[str(u)] += [str(v)]

In [ ]:
# Initialize pseudorandom number generator.
rand = Random(12345)

In [ ]:
def show_map(colors, show_png=False):
    df = pd.DataFrame(colors.items(), columns=['fips', 'color'], dtype=str)  # load as discrete values
    df = df[df['color'] != '0'].sort_values(by=['color'])

    fig = px.choropleth_mapbox(
        df,
        geojson=geo,
        locations='fips',
        color='color',
        color_discrete_map={str(i + 1): px.colors.qualitative.Plotly[i] for i in range(10)},
        mapbox_style='carto-positron',
        zoom=5.5,
        center = {"lat": 39.5, "lon": -111.55},
        width=500,
        height=500,
        opacity=0.8
    )
    fig.update_layout(
        margin=dict(r=0, t=0, l=0, b=0),
        showlegend=True
    )
    
    if show_png:
        display_png(Image(fig.to_image('png')))
    else:
        fig.show()

In [ ]:
show_map({county: 1 for county in counties}, show_png=False)

## Random Coloring

In [ ]:
num_colors = 5
colors = {county: rand.randint(1, num_colors) for county in counties}
show_map(colors)

## Greedy Coloring

In [ ]:
def run_step(state, show=False):
    running = True
    index = state['index']
    if index < len(state['counties']):
        county = state['counties'][index]
        used_colors = [state['colors'][nbr] for nbr in nbrs[county]]
        c = 1
        while c in used_colors:
            c += 1

        print(f'{index + 1:2d}: {county_names[county]:12s} -> color {c}')
        state['colors'][county] = c

        state['index'] += 1

    if show:
        show_map(state['colors'], show_png=False)
    return state['index'] < len(state['counties'])

In [ ]:
state = dict(colors=defaultdict(int), index=0, counties=sorted(counties))

In [ ]:
# Repeatedly run this cell.
run_step(state, show=True)

In [ ]:
state = dict(colors=defaultdict(int), index=0, counties=sorted(counties))
while True:
    if not run_step(state, show=False):
        break
show_map(state['colors'])